# Day 2: Classical CV Techniques
## CV Bootcamp 2024

Classical computer vision techniques that are still widely used today for feature detection, object recognition, and preprocessing.

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Create sample image
sample = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
# Add some geometric shapes for detection
cv2.rectangle(sample, (100, 100), (200, 200), (255, 255, 255), -1)
cv2.circle(sample, (400, 300), 50, (255, 255, 255), -1)
cv2.imwrite('sample_classical.jpg', sample)

image = cv2.imread('sample_classical.jpg')
print(f"Image loaded: {image.shape}")

## 1. Advanced Edge Detection

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# 1. Sobel Edge Detection (gradient-based)
sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)  # Horizontal edges
sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)  # Vertical edges

# Combine both directions
sobel_combined = np.sqrt(sobelx**2 + sobely**2)
sobel_combined = np.uint8(sobel_combined)

# 2. Laplacian Edge Detection (second derivative)
laplacian = cv2.Laplacian(gray, cv2.CV_64F)
laplacian = np.uint8(np.absolute(laplacian))

# 3. Canny (still the best)
canny = cv2.Canny(gray, 50, 150)

print("Edge detection methods applied")

In [ ]:
# Visualize different edge detectors
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

axes[0].imshow(gray, cmap='gray')
axes[0].set_title('Original Grayscale')

axes[1].imshow(sobelx, cmap='gray')
axes[1].set_title('Sobel X (Vertical Edges)')

axes[2].imshow(sobely, cmap='gray')
axes[2].set_title('Sobel Y (Horizontal Edges)')

axes[3].imshow(sobel_combined, cmap='gray')
axes[3].set_title('Sobel Combined')

axes[4].imshow(laplacian, cmap='gray')
axes[4].set_title('Laplacian')

axes[5].imshow(canny, cmap='gray')
axes[5].set_title('Canny')

for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 2. Contour Detection and Analysis

In [ ]:
# Convert to grayscale and threshold
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

# Find contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

print(f'Found {len(contours)} contours')

In [ ]:
# Draw all contours
result = image.copy()
cv2.drawContours(result, contours, -1, (0, 255, 0), 2)  # -1 means all contours

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
plt.title(f'All Contours ({len(contours)} found)')
plt.axis('off')
plt.show()

In [ ]:
# Filter contours by area
large_contours = [c for c in contours if cv2.contourArea(c) > 1000]
print(f'Large contours (area > 1000): {len(large_contours)}')

# Draw bounding boxes
result_boxes = image.copy()
for contour in large_contours:
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(result_boxes, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    # Get contour properties
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    
    # Add text
    cv2.putText(result_boxes, f'A:{int(area)}', (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(result_boxes, cv2.COLOR_BGR2RGB))
plt.title('Bounding Boxes with Area')
plt.axis('off')
plt.show()

## 3. Hough Transform - Line Detection

In [ ]:
# Create image with lines
line_image = np.zeros((480, 640, 3), dtype=np.uint8)
cv2.line(line_image, (100, 100), (500, 100), (255, 255, 255), 2)
cv2.line(line_image, (200, 200), (200, 400), (255, 255, 255), 2)
cv2.line(line_image, (300, 150), (450, 350), (255, 255, 255), 2)

gray_lines = cv2.cvtColor(line_image, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray_lines, 50, 150)

# Hough Line Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=50,
                        minLineLength=50, maxLineGap=10)

# Draw detected lines
result_lines = line_image.copy()
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(result_lines, (x1, y1), (x2, y2), (0, 255, 0), 3)
    print(f'Detected {len(lines)} lines')

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(line_image)
axes[0].set_title('Original')
axes[1].imshow(edges, cmap='gray')
axes[1].set_title('Edges')
axes[2].imshow(cv2.cvtColor(result_lines, cv2.COLOR_BGR2RGB))
axes[2].set_title('Detected Lines')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 4. Hough Transform - Circle Detection

In [ ]:
# Create image with circles
circle_image = np.zeros((480, 640, 3), dtype=np.uint8)
cv2.circle(circle_image, (200, 240), 80, (255, 255, 255), -1)
cv2.circle(circle_image, (450, 240), 60, (255, 255, 255), -1)

gray_circles = cv2.cvtColor(circle_image, cv2.COLOR_BGR2GRAY)
gray_circles = cv2.medianBlur(gray_circles, 5)

circles = cv2.HoughCircles(gray_circles, cv2.HOUGH_GRADIENT, dp=1, minDist=50,
                           param1=50, param2=30, minRadius=10, maxRadius=100)

# Draw detected circles
result_circles = circle_image.copy()
if circles is not None:
    circles = np.uint16(np.around(circles))
    for circle in circles[0, :]:
        cx, cy, radius = circle
        cv2.circle(result_circles, (cx, cy), radius, (0, 255, 0), 2)
        cv2.circle(result_circles, (cx, cy), 2, (0, 0, 255), 3)
    print(f'Detected {len(circles[0])} circles')

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(circle_image)
axes[0].set_title('Original')
axes[1].imshow(cv2.cvtColor(result_circles, cv2.COLOR_BGR2RGB))
axes[1].set_title('Detected Circles')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 5. Feature Detection with ORB

In [ ]:
# Create ORB detector
orb = cv2.ORB_create(nfeatures=500)

# Detect keypoints and compute descriptors
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
keypoints, descriptors = orb.detectAndCompute(gray, None)

print(f'Found {len(keypoints)} keypoints')
if descriptors is not None:
    print(f'Descriptor shape: {descriptors.shape}')

In [ ]:
# Draw keypoints
result_orb = cv2.drawKeypoints(image, keypoints, None,
                               flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

plt.figure(figsize=(12, 8))
plt.imshow(cv2.cvtColor(result_orb, cv2.COLOR_BGR2RGB))
plt.title(f'ORB Keypoints ({len(keypoints)} detected)')
plt.axis('off')
plt.show()

## 6. Template Matching

In [ ]:
# Create a template (small region to find)
template = gray[100:200, 100:200]
template_h, template_w = template.shape

# Perform template matching
result_match = cv2.matchTemplate(gray, template, cv2.TM_CCOEFF_NORMED)

# Find best match location
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result_match)

print(f'Best match confidence: {max_val:.4f}')
print(f'Match location: {max_loc}')

In [ ]:
# Draw rectangle around match
top_left = max_loc
bottom_right = (top_left[0] + template_w, top_left[1] + template_h)

result_template = cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB)
cv2.rectangle(result_template, top_left, bottom_right, (0, 255, 0), 2)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
axes[0].set_title('Original')
axes[1].imshow(template, cmap='gray')
axes[1].set_title('Template')
axes[2].imshow(result_template)
axes[2].set_title(f'Match Found (conf={max_val:.2f})')
for ax in axes:
    ax.axis('off')
plt.tight_layout()
plt.show()

## 7. Face Detection with Haar Cascades

In [ ]:
# Load pre-trained face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
)

# Load eye detector
eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_eye.xml'
)

print("Haar cascades loaded successfully")
print(f"Face cascade empty: {face_cascade.empty()}")
print(f"Eye cascade empty: {eye_cascade.empty()}")

In [ ]:
# Create a simple test image (for demonstration)
# In practice, you would load an actual face image
test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
gray_test = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)

# Detect faces
faces = face_cascade.detectMultiScale(
    gray_test,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30)
)

print(f'Found {len(faces)} faces')

# Draw rectangles around faces
result_faces = test_image.copy()
for (x, y, w, h) in faces:
    cv2.rectangle(result_faces, (x, y), (x + w, y + h), (255, 0, 0), 2)
    cv2.putText(result_faces, 'Face', (x, y-10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

print("Face detection complete")

## 8. Real-Time Face Detection Function

In [ ]:
def detect_faces_webcam():
    """
    Real-time face detection using webcam.
    Press 'q' to quit, 's' to save current frame.
    
    NOTE: This function requires a webcam and will open a window.
    Comment out or skip if running headless.
    """
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print('Error: Could not open webcam')
        return
    
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    
    frame_count = 0
    print('Starting face detection... Press q to quit, s to save frame')
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print('Error: Failed to capture frame')
            break
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        faces = face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
        )
        
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, 'Face', (x, y - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        
        info_text = f'Faces: {len(faces)} | Press q to quit, s to save'
        cv2.putText(frame, info_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv2.imshow('Face Detection', frame)
        
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('q'):
            break
        elif key == ord('s'):
            filename = f'face_detection_{frame_count}.jpg'
            cv2.imwrite(filename, frame)
            print(f'Saved {filename}')
            frame_count += 1
    
    cap.release()
    cv2.destroyAllWindows()
    print('Face detection completed')

# Uncomment to run (requires webcam):
# detect_faces_webcam()
print("Face detection function defined (uncomment to run with webcam)")

## Summary

You've learned:
- ✓ Advanced edge detection (Sobel, Laplacian, Canny)
- ✓ Contour detection and analysis
- ✓ Hough Transform for line and circle detection
- ✓ ORB feature detection and matching
- ✓ Template matching for object detection
- ✓ Haar Cascade face detection
- ✓ Real-time webcam applications

**Key Takeaway:** Classical CV techniques are fast, interpretable, and still useful for many applications!